In [8]:
import pandas as pd 

#해당 링크는 한국거래소에서 상장법인목록을 엑셀로 다운로드하는 링크입니다.
#다운로드와 동시에 Pandas에 excel 파일이 load가 되는 구조입니다.
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0] 
#stock_code.head()

# 데이터에서 정렬이 따로 필요하지는 않지만 테스트겸 Pandas sort_values를 이용하여 정렬을 시도해봅니다.
stock_code.sort_values(['상장일'], ascending=True)

# 필요한 것은 "회사명"과 "종목코드" 이므로 필요없는 column들은 제외
stock_code = stock_code[['회사명', '종목코드']] 

# 한글 컬럼명을 영어로 변경 
stock_code = stock_code.rename(columns={'회사명': 'company', '종목코드': 'code'}) 
#stock_code.head()

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
stock_code.code = stock_code.code.map('{:06d}'.format) 

In [9]:
# LG화학의 일별 시세 url 가져오기 
company='LG화학' 
code = stock_code[stock_code.company==company].code.values[0].strip() ## strip() : 공백제거
page = 1

url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)     
url = '{url}&page={page}'.format(url=url, page=page)
print(url)
df = pd.read_html(url, header=0)[0]
df.head()

http://finance.naver.com/item/sise_day.nhn?code=051910&page=1


,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.10.08,692000.0,12000.0,684000.0,698000.0,682000.0,541159.0
2,2020.10.07,680000.0,6000.0,673000.0,680000.0,667000.0,294545.0
3,2020.10.06,674000.0,15000.0,659000.0,680000.0,654000.0,451956.0
4,2020.10.05,659000.0,5000.0,670000.0,670000.0,648000.0,443499.0


In [10]:
company='LG화학' 
code = stock_code[stock_code.company==company].code.values[0].strip() ## strip() : 공백제거

df = pd.DataFrame()
for page in range(1,21):
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)     
    url = '{url}&page={page}'.format(url=url, page=page)
    print(url)
    df = df.append(pd.read_html(url, header=0)[0], ignore_index=True)

http://finance.naver.com/item/sise_day.nhn?code=051910&page=1
http://finance.naver.com/item/sise_day.nhn?code=051910&page=2
http://finance.naver.com/item/sise_day.nhn?code=051910&page=3
http://finance.naver.com/item/sise_day.nhn?code=051910&page=4
http://finance.naver.com/item/sise_day.nhn?code=051910&page=5
http://finance.naver.com/item/sise_day.nhn?code=051910&page=6
http://finance.naver.com/item/sise_day.nhn?code=051910&page=7
http://finance.naver.com/item/sise_day.nhn?code=051910&page=8
http://finance.naver.com/item/sise_day.nhn?code=051910&page=9
http://finance.naver.com/item/sise_day.nhn?code=051910&page=10
http://finance.naver.com/item/sise_day.nhn?code=051910&page=11
http://finance.naver.com/item/sise_day.nhn?code=051910&page=12
http://finance.naver.com/item/sise_day.nhn?code=051910&page=13
http://finance.naver.com/item/sise_day.nhn?code=051910&page=14
http://finance.naver.com/item/sise_day.nhn?code=051910&page=15
http://finance.naver.com/item/sise_day.nhn?code=051910&page=16
h

In [11]:
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() 

# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 

# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 

# 상위 5개 데이터 확인 
df.head()

,date,close,diff,open,high,low,volume
298,2019-12-17,305500,3500,306500,308000,305000,176726
297,2019-12-18,307000,1500,307000,307000,298000,207013
296,2019-12-19,309000,2000,304500,309500,302500,110138
295,2019-12-20,314500,5500,310000,315500,309500,248130
294,2019-12-23,313000,1500,314000,315000,311000,121761


In [12]:
import matplotlib.pyplot as plt
# 필요한 모듈 import 하기 
import plotly
import plotly.graph_objects as go
import plotly.express as px

# %matplotlib inline 은 jupyter notebook 사용자용 - jupyter notebook 내에 그래프가 그려지게 한다.
%matplotlib inline

In [13]:
fig = px.line(df, x='date', y='close', title='{}의 종가(close) Time Series'.format(company))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()
fig.write_html("file.html")